In [1]:
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv


c:\Users\Raghavan\Documents\BudgetBuddy\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [5]:
os.chdir('../')

In [94]:
def get_pdf_text(pdf_docs):
    text = ''
    for pdf in os.listdir(pdf_docs):
        pdf_reader = PdfReader(os.path.join(pdf_docs,pdf))
        for page in pdf_reader.pages:
            text+=page.extract_text()
    return text

book1 = r'references'
texts = get_pdf_text(book1)


' be \nentered in the revenue column.  \nNow as an expense . You are left with 80% of thy \nearnings. How to divide? Some experts recommended to do so:   \n 28 a) 50% spend all the necessary expenditures (food, \nutilities, transportation  costs, clothing, etc.),  \nb) 30% of the things like - Internet, cable TV, restaurants, \ntickets to the theater s, concerts, books, hobbies, etc. t. \nt.). \nDistribution of a particular group are parts of the budget \nso as you need , such as utilities, food, etc.  just like do -well \nwith a second group of budget.  \nThen you start to compile forecasts, articles spending \nand what you plan to buy in the near future and how much it \npays around. Try spending zoom in revenue. So, you gradually \nlearn to feel, where you can spend  more money, but where you \nneed to save and why.  \nEveryone has a different understanding of quality of \nlife. Som e, maybe you can not imagine their life without a night \nclub visits and restaurants, but solid, per

In [39]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 10000,chunk_overlap = 1000)
    chunks = text_splitter.split_text(text)
    return chunks

chunks = get_text_chunks(texts)

In [ ]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks,embedding = embeddings)
    vector_store.save_local("faiss_index")
    print("vector store successfully saved!!")

vector_store = get_vector_store(chunks)


In [45]:
retriever = vector_store.as_retriever()

In [98]:
expenses = {
    'Rent':15000,
    'food': 4000,
    'others': 3000
}
savings = 2000
name = 'shinji'
user_persona = f"Username {name} User's todays Spendings {expenses.items()} User's desired savings for this month {savings}"
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """You are a financial expert specialized in personal finance, integrated into the BudgetBuddy app. 
Users can ask you any questions about their finances, including personalized queries related to their spending, saving habits, and optimal budget.  

Retrieve relevant information from the context and their persona (including their tracked spending, cumulative expenses, and savings goals) to provide **accurate and personalized answers** to their queries.
 If the user asks for general financial advice (e.g., "How to invest?" or "How to increase income?"), provide comprehensive,
   actionable advice based on general financial knowledge and the context available.  

If the question is unclear or unrelated to personal finance, politely request clarification or provide a 
generic overview of the topic if it fits within the financial domain. Always prioritize politeness and motivation in your tone, 
ensuring that users feel supported and guided in their financial journey.  
 
You can have a reference on the following context whenever needed (you dont have to specify the references just say based on my knowledge)
and the information given is :
{context}

Question: {question}
""" + user_persona
prompt = ChatPromptTemplate.from_template(template)
model = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs,"question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

response = chain.invoke("Provide me a feedback on today's spending pattern?")


In [ ]:
def load_vectorstore():
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
    return vector_store.as_retriever()

retriever = load_vectorstore()


VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022EB80647C0>, search_kwargs={})

In [105]:
from utils.rag_utils import prompt_template_chat,format_docs
template = prompt_template_chat()+user_persona
chain = (
    {"context": retriever | format_docs,"question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
query = "How can I earn passive income. I don't have any external skillset. Suggest me a passive income option"
response = chain.invoke(query)


In [107]:
print(response)

Hello shinji!  Based on my knowledge, achieving truly passive income without any existing skills requires significant upfront effort and may not generate substantial returns immediately.  The concept of "passive income" is often a misnomer; most income streams require ongoing maintenance and attention, even if not directly involving your time on a daily basis.

Given your current spending (Rent: 15000, Food: 4000, Others: 3000) and a savings goal of 2000, you're already allocating a portion of your income to savings.  Before pursuing passive income, focus on building a solid financial foundation.  This includes:

1. **Emergency Fund:**  Ensure you have at least 3-6 months' worth of living expenses saved in a readily accessible account. This protects against unexpected events and prevents you from needing to dip into long-term investments.

2. **Debt Management:** Prioritize paying off any high-interest debt (credit cards, personal loans) before seeking passive income streams.  The inte

In [ ]:
dict1 = {"v":1,"u":2,"w":3}
dict1

TypeError: unhashable type: 'list'